# Tutorial
* https://www.youtube.com/watch?v=SG6jdlBx_vQ
    
* https://github.com/ZWMiller/nlp_pipe_manager/tree/master/nlp_pipeline_manager

* https://github.com/ZWMiller/nlp_pipe_manager/blob/master/nlp_pipeline_manager/pipeline_demo.ipynb

# Building a class to manage our NLP pipelines

Because it's such a pain to manage all the permutations of NLP cleaners/tokenizers/vectorizers/stemmers/etc, we're going to build a class that takes all of those pieces in and manages the pipelines for us.

In [1]:
import numpy as np
import sklearn
import matplotlib
import pandas as pd
import sklearn
import sys
import nltk

libraries = (('Matplotlib', matplotlib), ('Numpy', np), ('Pandas', pd), ('NLTK', nltk), ('sklearn',sklearn))

print("Python Version:", sys.version, '\n')
for lib in libraries:
    print('{0} Version: {1}'.format(lib[0], lib[1].__version__))

Python Version: 3.6.0 (default, Jun 23 2017, 18:28:15) 
[GCC 4.2.1 Compatible Apple LLVM 8.1.0 (clang-802.0.42)] 

Matplotlib Version: 3.0.2
Numpy Version: 1.17.0
Pandas Version: 0.25.0
NLTK Version: 3.2.4
sklearn Version: 0.21.3


In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import math
import scipy
%matplotlib inline
plt.style.use('seaborn')

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle

class nlp_preprocessor:
   
    def __init__(self, vectorizer=CountVectorizer(), tokenizer=None, cleaning_function=None, 
                 stemmer=None, model=None):
        """
        A class for pipelining our data in NLP problems. The user provides a series of 
        tools, and this class manages all of the training, transforming, and modification
        of the text data.
        ---
        Inputs:
        vectorizer: the model to use for vectorization of text data
        tokenizer: The tokenizer to use, if none defaults to split on spaces
        cleaning_function: how to clean the data, if None, defaults to the in built class
        """
        if not tokenizer:
            tokenizer = self.splitter
        if not cleaning_function:
            cleaning_function = self.clean_text
        self.stemmer = stemmer
        self.tokenizer = tokenizer
        self.model = model
        self.cleaning_function = cleaning_function
        self.vectorizer = vectorizer
        self._is_fit = False
        
    def splitter(self, text):
        """
        Default tokenizer that splits on spaces naively
        """
        return text.split(' ')
        
    def clean_text(self, text, tokenizer, stemmer):
        """
        A naive function to lowercase all works can clean them quickly.
        This is the default behavior if no other cleaning function is specified
        """
        cleaned_text = []
        for post in text:
            cleaned_words = []
            for word in tokenizer(post):
                low_word = word.lower()
                if stemmer:
                    low_word = stemmer.stem(low_word)
                cleaned_words.append(low_word)
            cleaned_text.append(' '.join(cleaned_words))
        return cleaned_text
    
    def fit(self, text):
        """
        Cleans the data and then fits the vectorizer with
        the user provided text
        """
        clean_text = self.cleaning_function(text, self.tokenizer, self.stemmer)
        self.vectorizer.fit(clean_text)
        self._is_fit = True
        
    def transform(self, text):
        """
        Cleans any provided data and then transforms the data into
        a vectorized format based on the fit function. Returns the
        vectorized form of the data.
        """
        if not self._is_fit:
            raise ValueError("Must fit the models before transforming!")
        clean_text = self.cleaning_function(text, self.tokenizer, self.stemmer)
        return self.vectorizer.transform(clean_text)
    
    def save_pipe(self, filename):
        """
        Writes the attributes of the pipeline to a file
        allowing a pipeline to be loaded later with the
        pre-trained pieces in place.
        """
        if type(filename) != str:
            raise TypeError("filename must be a string")
        pickle.dump(self.__dict__, open(filename+".mdl",'wb'))
        
    def load_pipe(self, filename):
        """
        Writes the attributes of the pipeline to a file
        allowing a pipeline to be loaded later with the
        pre-trained pieces in place.
        """
        if type(filename) != str:
            raise TypeError("filename must be a string")
        if filename[-4:] != '.mdl':
            filename += '.mdl'
        self.__dict__ = pickle.load(open(filename,'rb'))

In [ ]:
text.lemma

## Now let's test the model with the defaults

In [4]:
corpus = ['BOB the builder', 'is a strange', 'caRtoon type thing']

In [5]:
nlp = nlp_preprocessor()

In [6]:
nlp.fit(corpus)

In [7]:
nlp.transform(corpus).toarray()

array([[1, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 1]])

In [8]:
pd.DataFrame(nlp.transform(corpus).toarray(), columns=nlp.vectorizer.get_feature_names())

,bob,builder,cartoon,is,strange,the,thing,type
0,1,1,0,0,0,1,0,0
1,0,0,0,1,1,0,0,0
2,0,0,1,0,0,0,1,1


## What if we want to swap pieces in?

In [9]:
def new_clean_text(text, tokenizer, stemmer):
    """
    A naive function to lowercase all works can clean them quickly.
    This is the default behavior if no other cleaning function is specified
    """
    cleaned_text = []
    for post in text:
        cleaned_words = []
        for word in tokenizer(post):
            low_word = word.lower()
            if low_word in ['builder']: # remove the word builder
                continue
            if stemmer:
                low_word = stemmer.stem(low_word)
            cleaned_words.append(low_word)
        cleaned_text.append(' '.join(cleaned_words))
    return cleaned_text

In [10]:
from nltk.stem import PorterStemmer

nlp2 = nlp_preprocessor(cleaning_function=new_clean_text, vectorizer=CountVectorizer(lowercase=False), 
                        stemmer=PorterStemmer())

In [11]:
nlp2.fit(corpus)
nlp2.vectorizer.get_feature_names()

['bob', 'cartoon', 'is', 'strang', 'the', 'thing', 'type']

In [12]:
pd.DataFrame(nlp2.transform(corpus).toarray(), columns=nlp2.vectorizer.get_feature_names())

,bob,cartoon,is,strang,the,thing,type
0,1,0,0,0,1,0,0
1,0,0,1,1,0,0,0
2,0,1,0,0,0,1,1


## What about using TF-IDF instead?

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

nlp3 = nlp_preprocessor(cleaning_function=new_clean_text, vectorizer=TfidfVectorizer(lowercase=False))

In [14]:
nlp3.fit(corpus)
nlp3.vectorizer.get_feature_names()
pd.DataFrame(nlp3.transform(corpus).toarray(), columns=nlp3.vectorizer.get_feature_names())

,bob,cartoon,is,strange,the,thing,type
0,0.707107,0.00000,0.000000,0.000000,0.707107,0.00000,0.00000
1,0.000000,0.00000,0.707107,0.707107,0.000000,0.00000,0.00000
2,0.000000,0.57735,0.000000,0.000000,0.000000,0.57735,0.57735


# So what? Let's use some real data to try some different modeling approaches

In [18]:
from sklearn import datasets

categories = ['alt.atheism', 'comp.graphics', 'rec.sport.baseball']
ng_train = datasets.fetch_20newsgroups(subset='train', 
                                       categories=categories, 
                                       remove=('headers', 
                                               'footers', 'quotes'))
ng_train_data = ng_train.data
ng_train_targets = ng_train.target

ng_test = datasets.fetch_20newsgroups(subset='test', 
                                       categories=categories, 
                                       remove=('headers', 
                                               'footers', 'quotes'))

ng_test_data = ng_test.data
ng_test_targets = ng_test.target

URLError: <urlopen error unknown url type: https>

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import PorterStemmer

nlp = nlp_preprocessor(stemmer=PorterStemmer())
nlp2 = nlp_preprocessor(vectorizer=CountVectorizer(lowercase=False))
nlp3 = nlp_preprocessor(cleaning_function=new_clean_text, vectorizer=TfidfVectorizer(lowercase=False))
nlp_chains = [nlp, nlp2, nlp3]

for ix, chain in enumerate(nlp_chains):
    nb = MultinomialNB()
    chain.fit(ng_train_data)
    train_data = chain.transform(ng_train_data)
    test_data = chain.transform(ng_test_data)
    nb.fit(train_data, ng_train_targets)
    accuracy = nb.score(test_data, ng_test_targets)
    print("Chain {}: {}".format(ix, accuracy))

## Summary

This allows us to sweep all of the preprocessing into a class where we can control the pieces and parts that go in, and can see what comes out. If we wanted to, we could even add a model into the class as well and put the whole pipe into a single class that manages all of our challenges. In this case, we've left it outside for demo purposes. This also saves all of the pieces together, so we can just pickle a class object and that will keep the whole structure of our models together - such as the vectorizer and the stemmer we used, as well as the cleaning routine, so we don't lose any of the pieces if we want to run it on new data later.

# Adding a model to the mix

Depending on the type of model we want to build, we'll need to wrap the preprocessing class a little bit differently for the specific case. For example, if we're doing supervised learning, we'll want a `predict` method. If we're doing topic modeling, we'll want a `transform` method. To make that happen, I'll show a few examples below that wrap around the preprocessing class to make the most of it. 

#### Supervised: Classification

Here we'll write a class to predict a class given the text of the document. 

In [16]:
class supervised_nlp:
    
    def __init__(self, model, preprocessing_pipeline=None):
        """
        A pipeline for doing supervised nlp. Expects a model and creates
        a preprocessing pipeline if one isn't provided.
        """
        self.model = model
        self._is_fit = False
        if not preprocessing_pipeline:
            self.preprocessor = nlp_preprocessor()
        else:
            self.preprocessor = preprocessing_pipeline
        
    def fit(self, X, y):
        """
        Trains the vectorizer and model together using the 
        users input training data.
        """
        self.preprocessor.fit(X)
        train_data = self.preprocessor.transform(X)
        self.model.fit(train_data, y)
        self._is_fit = True
    
    def predict(self, X):
        """
        Makes a prediction on the data provided by the users using the 
        preprocessing pipeline and provided model.
        """
        if not self._is_fit:
            raise ValueError("Must fit the models before transforming!")
        test_data = self.preprocessor.transform(X)
        preds = self.model.predict(test_data)
        return preds
    
    def score(self, X, y):
        """
        Returns the accuracy for the model after using the trained
        preprocessing pipeline to prepare the data.
        """
        test_data = self.preprocessor.transform(X)
        return self.model.score(test_data, y)
    
    def save_pipe(self, filename):
        """
        Writes the attributes of the pipeline to a file
        allowing a pipeline to be loaded later with the
        pre-trained pieces in place.
        """
        if type(filename) != str:
            raise TypeError("filename must be a string")
        pickle.dump(self.__dict__, open(filename+".mdl",'wb'))
        
    def load_pipe(self, filename):
        """
        Writes the attributes of the pipeline to a file
        allowing a pipeline to be loaded later with the
        pre-trained pieces in place.
        """
        if type(filename) != str:
            raise TypeError("filename must be a string")
        if filename[-4:] != '.mdl':
            filename += '.mdl'
        self.__dict__ = pickle.load(open(filename,'rb'))

In [17]:
nlp_pipe = supervised_nlp(MultinomialNB(), nlp)
nlp_pipe.fit(ng_train_data, ng_train_targets)
nlp_pipe.score(ng_test_data, ng_test_targets)

NameError: name 'MultinomialNB' is not defined

Swap out the model for something different.

In [19]:
from sklearn.svm import LinearSVC

nlp_pipe = supervised_nlp(LinearSVC(), nlp)
nlp_pipe.fit(ng_train_data, ng_train_targets)
nlp_pipe.score(ng_test_data, ng_test_targets)

0.8262443438914027

#### Unsupervised: Topic Modeling

We don't want to make a prediction with this example, simply to find topics and have the ability to cast our data into the "topic space" from the "word space." With this in mind, we'll add a transform feature and also the ability to print out the topics.

In [20]:
class topic_modeling_nlp:
    
    def __init__(self, model, preprocessing_pipeline=None):
        """
        A pipeline for doing supervised nlp. Expects a model and creates
        a preprocessing pipeline if one isn't provided.
        """
        self.model = model
        self._is_fit = False
        if not preprocessing_pipeline:
            self.preprocessor = nlp_preprocessor()
        else:
            self.preprocessor = preprocessing_pipeline
        
    def fit(self, X):
        """
        Trains the vectorizer and model together using the 
        users input training data.
        """
        self.preprocessor.fit(X)
        train_data = self.preprocessor.transform(X)
        self.model.fit(train_data)
        self._is_fit = True
    
    def transform(self, X):
        """
        Makes a prediction on the data provided by the users using the 
        preprocessing pipeline and provided model.
        """
        if not self._is_fit:
            raise ValueError("Must fit the models before transforming!")
        test_data = self.preprocessor.transform(X)
        preds = self.model.transform(test_data)
        return preds
    
    def print_topics(self, num_words=10):
        """
        A function to print out the top words for each topic
        """
        feat_names = self.preprocessor.vectorizer.get_feature_names()
        for topic_idx, topic in enumerate(self.model.components_):
            message = "Topic #%d: " % topic_idx
            message += " ".join([feat_names[i]
                                 for i in topic.argsort()[:-num_words - 1:-1]])
            print(message)
            
    def save_pipe(self, filename):
        """
        Writes the attributes of the pipeline to a file
        allowing a pipeline to be loaded later with the
        pre-trained pieces in place.
        """
        if type(filename) != str:
            raise TypeError("filename must be a string")
        pickle.dump(self.__dict__, open(filename+".mdl",'wb'))
        
    def load_pipe(self, filename):
        """
        Writes the attributes of the pipeline to a file
        allowing a pipeline to be loaded later with the
        pre-trained pieces in place.
        """
        if type(filename) != str:
            raise TypeError("filename must be a string")
        if filename[-4:] != '.mdl':
            filename += '.mdl'
        self.__dict__ = pickle.load(open(filename,'rb'))

In [21]:
from sklearn.decomposition import TruncatedSVD

cv = CountVectorizer(stop_words='english', token_pattern='\\b[a-z][a-z]+\\b')
cleaning_pipe = nlp_preprocessor(vectorizer=cv)
topic_chain = topic_modeling_nlp(TruncatedSVD(n_components=15), preprocessing_pipeline=cleaning_pipe)

In [22]:
topic_chain.fit(ng_train_data)
topic_chain.transform(ng_train_data).shape

(1661, 15)

In [23]:
topic_chain.print_topics()

Topic #0: jpeg image edu file graphics gif images format color pub
Topic #1: edu graphics pub data mail ray ftp send com objects
Topic #2: jesus god atheists matthew people atheism does religious said religion
Topic #3: image data processing analysis software available display tools tool user
Topic #4: jesus matthew prophecy messiah psalm isaiah david said lord israel
Topic #5: argument fallacy conclusion example true argumentum ad premises false valid
Topic #6: data available ftp sgi grass vertex pci motecc model info
Topic #7: game year don good think hit won runs team home
Topic #8: god posting subject response typical information universe einstein bush evidence
Topic #9: den radius double theta sqrt pi sin rtheta pole pt
Topic #10: program read menu bits display change file pressing want don
Topic #11: lost program won cubs atheism game menu display bits home
Topic #12: game runs second hit run cubs graphics home win sunday
Topic #13: atheism alt faq send edu usenet news files otis

Swap out the model for something different.

In [24]:
from sklearn.decomposition import LatentDirichletAllocation
topic_chain = topic_modeling_nlp(LatentDirichletAllocation(n_components=15), preprocessing_pipeline=cleaning_pipe)

In [25]:
topic_chain.fit(ng_train_data)
topic_chain.transform(ng_train_data).shape
topic_chain.print_topics()

/Users/zachariahmiller/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


Topic #0: cornerstone pappas nis dualpage dollar tour karlin eastwick rawley mcnally
Topic #1: corel tune pens bone trimming dykstra reprints highway qcr packs
Topic #2: enviroleague youth mission organizations markus true adult max pointer abekas
Topic #3: image graphics edu file jpeg data files software use images
Topic #4: edu cobb hou jewish polygon problem illinois schwartz sigkids pay
Topic #5: graeme computer radiosity pp curves cubic bezier stephan generation dtax
Topic #6: god people atheism does atheists jesus religion don believe argument
Topic #7: sex sea tek said vice bronx com ico bob away
Topic #8: won cubs lost york new team edu san sox reds
Topic #9: den col int radius row war value bombing theta hussein
Topic #10: text copy texts septuagint masoretic parody ot passages various toilet
Topic #11: erickson cage animals temporarily syllogism spell cch cold products polygoon
Topic #12: alomar average league players baerga double player rbi ab slg
Topic #13: xxxx cruel anti